In [5]:
!pip install mediapipe opencv-python

In [1]:
import mediapipe as mp
import numpy as np
import time
import cv2
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

<h1>Test

In [3]:
# cap=cv2.VideoCapture('KneeBendVideo.mp4')
# while True:
#     ret,frame=cap.read()
#     cv2.imshow("Mediapipe Feed",frame)
    
#     if cv2.waitKey(25)&0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [2]:
# def calculate_angle(a,b,c):
#     a=np.array(a) #First
#     b=np.array(b) #Mid
#     c=np.array(c) #End

#     radians = np.arctan2(c[1]-b[1],c[0]-b[0])- np.arctan2(a[1]-b[1],a[0]-b[0])
#     angle=np.abs(radians*180.0/np.pi)

#     if angle>180.0:
#         angle=360-angle
    
#     return angle

In [33]:
import mediapipe as mp
import numpy as np
import cv2
import time

mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


cap = cv2.VideoCapture('KneeBendVideo.mp4')

stage = None
start=0
end=0
timeTaken=0
prevtimeTaken=0
flag=False
rep=0
k=1

# Setup Mediapipe Instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # image.flags.writeable=False

        # Make Detection
        results = pose.process(image)

        # Recolor to BGR
        # image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get Coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate Angle
            angle = calculate_angle(hip, knee, ankle)

            # Visualize Angle
            cv2.putText(image, str(angle),
                        tuple(np.multiply(knee, [854, 640]).astype(int)),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

            # print(landmarks)


            if angle < 140:
                end = time.time()
                k=1
                if start > 0:
                    timeTaken = end-start
                else:
                    pass
            if angle > 140:
                if timeTaken<8 and k==0:
                    cv2.putText(image, 'Keep Your Knee Bent', (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
                flag=True
                k=1
                timeTaken=0
                start = time.time()

            if timeTaken > 8 and flag:
                start = time.time()
                rep+=1
                flag=False
            elif flag==False:
                k=0        
        except:
            pass

        cv2.rectangle(image, (0, 0), (175, 73), (245, 117, 16), -1)

        # Rep Data
        cv2.putText(image,'REPS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(rep),(10,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)

        #if k==0:
            # cv2.putText(image, 'Keep Your Knee Bent', (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Render Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [30]:


start=time.time()

while True:
    if time.time()-start>=8:
        print(2)
        start=time.time()

2


KeyboardInterrupt: 